In [7]:
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels
import hvplot.xarray

In [8]:
station = "Hot"

## Load DATA


### Observations


In [9]:
data_obs = xr.open_dataset(f"../data/1_products/{station}_obs_zoo_climato_monthly_2002_2015.zarr", engine="zarr").load()
data_obs

<xarray.Dataset> Size: 6kB
Dimensions:            (time: 155, latitude: 1, longitude: 1, layer: 1)
Coordinates:
  * latitude           (latitude) float64 8B 22.75
  * layer              (layer) int64 8B 0
  * longitude          (longitude) int64 8B -158
  * time               (time) datetime64[ns] 1kB 2002-02-15 ... 2014-12-15
Data variables:
    day                (time, latitude, longitude, layer) float64 1kB 1.473 ....
    day_lowess_0.15    (time, latitude, longitude, layer) float64 1kB 1.506 ....
    night              (time, latitude, longitude, layer) float64 1kB 2.552 ....
    night_lowess_0.15  (time, latitude, longitude, layer) float64 1kB 2.533 ....
Attributes:
    desc:     Aggregated and smoothed observations using lowess filtre.

In [10]:
time_start = data_obs.time.data[0]
time_end = data_obs.time.data[-1]

### CMEMS


In [11]:
data_cmems = xr.open_dataset(f"../data/1_products/{station}_cmems.zarr", engine="zarr").load()
data_cmems = data_cmems.sel(time=slice(time_start, time_end))
data_cmems

<xarray.Dataset> Size: 825kB
Dimensions:              (latitude: 1, longitude: 1, time: 4687, depth: 3)
Coordinates:
  * depth                (depth) int64 24B 0 1 2
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * time                 (time) datetime64[ns] 37kB 2002-02-15 ... 2014-12-15
Data variables: (12/13)
    T                    (latitude, longitude, time, depth) float64 112kB 22....
    U                    (latitude, longitude, time, depth) float64 112kB -0....
    V                    (latitude, longitude, time, depth) float64 112kB -0....
    mnkc_epi             (latitude, longitude, time) float64 37kB 0.4676 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 37kB 1.018 ... 1...
    mnkc_lmeso           (latitude, longitude, time) float64 37kB 1.859 ... 2...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 37kB 0.6909 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 37kB 0.4053 ... ...
    npp                  (latitude, longitude, time) float64 37kB 387.7 ... 3...
    pelagic_layer_depth  (latitude, longitude, time, depth) float64 112kB 110...
    zeu                  (latitude, longitude, time) float64 37kB 73.93 ... 7...
    zooc                 (latitude, longitude, time) float64 37kB 0.4146 ... ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-10-17
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Global ocean low and mid trophic levels biomass content hin...

## Climato


In [12]:
for i in range(1, 13):
    print(
        f"Month {i} : ",
        set(
            pd.DataFrame({"month": data_cmems.indexes["time"].month, "day": data_cmems.indexes["time"].day})
            .query(f"month == {i}")["day"]
            .unique()
        ),
    )

Month 1 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 2 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29}
Month 3 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 4 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 5 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 6 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30}
Month 7 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}
Month 8 :  {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31}


In [13]:
data_cmems_climato = data_cmems.groupby("time.dayofyear").mean()

## Expand to original time series


In [14]:
climato_merged = {}

for var in data_cmems.data_vars:
    if "depth" in list(data_cmems[var].coords):
        data = []
        for depth in data_cmems.depth.data:
            tab_1 = data_cmems.time.dt.dayofyear.to_series()
            tab_2 = data_cmems_climato[var].sel(depth=depth).to_series()
            data.append(pd.merge(tab_1, tab_2, on="dayofyear", how="left")[var])
        climato_merged[var] = xr.DataArray(
            data, dims=("depth", "time"), coords={"depth": data_cmems.depth.data, "time": data_cmems.time.data}
        )
    else:
        tab_1 = data_cmems.time.dt.dayofyear.to_series()
        tab_2 = data_cmems_climato[var].to_series()
        climato_merged[var] = xr.DataArray(
            pd.merge(tab_1, tab_2, on="dayofyear", how="left")[var],
            dims=("time"),
            coords={"time": data_cmems.time.data},
        )

climato_merged = xr.Dataset(climato_merged)

climato_merged = climato_merged.expand_dims({"latitude": data_cmems.latitude, "longitude": data_cmems.longitude})
for var in climato_merged:
    climato_merged[var].attrs = data_cmems[var].attrs
for coord in climato_merged.coords:
    climato_merged[coord].attrs = data_cmems[coord].attrs
climato_merged

<xarray.Dataset> Size: 825kB
Dimensions:              (latitude: 1, longitude: 1, depth: 3, time: 4687)
Coordinates:
  * latitude             (latitude) float64 8B 22.75
  * longitude            (longitude) int64 8B -158
  * depth                (depth) int64 24B 0 1 2
  * time                 (time) datetime64[ns] 37kB 2002-02-15 ... 2014-12-15
Data variables: (12/13)
    T                    (latitude, longitude, depth, time) float64 112kB 22....
    U                    (latitude, longitude, depth, time) float64 112kB -0....
    V                    (latitude, longitude, depth, time) float64 112kB 0.0...
    mnkc_epi             (latitude, longitude, time) float64 37kB 0.3534 ... ...
    mnkc_hmlmeso         (latitude, longitude, time) float64 37kB 0.8613 ... ...
    mnkc_lmeso           (latitude, longitude, time) float64 37kB 2.074 ... 2...
    ...                   ...
    mnkc_mumeso          (latitude, longitude, time) float64 37kB 0.6117 ... ...
    mnkc_umeso           (latitude, longitude, time) float64 37kB 0.4642 ... ...
    npp                  (latitude, longitude, time) float64 37kB 314.7 ... 3...
    pelagic_layer_depth  (latitude, longitude, depth, time) float64 112kB 118...
    zeu                  (latitude, longitude, time) float64 37kB 79.22 ... 7...
    zooc                 (latitude, longitude, time) float64 37kB 0.3151 ... ...

## Plot


In [15]:
climato_merged.mean(["latitude", "longitude"]).hvplot.line(
    x="time",
    groupby="depth",
    title=f"Climato {station} - CMEMS T",
)

BokehModel(combine_events=True, render_bundle={'docs_json': {'1b266bec-8734-49cd-a87f-d0bf89df46e0': {'version…

## Export


In [16]:
climato_merged.to_zarr(
    f"../data/1_products/{station}_cmems_climato_{pd.Timestamp(time_start).year}_{pd.Timestamp(time_end).year}.zarr",
    mode="w",
)